### Import modules

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from  matplotlib import pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

### Preprocessing

In [ ]:
batch_size =128
img_height = 224
img_width = 224

directory = "../input/emd-hht/emd_image"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  directory,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='binary',
  image_size=(img_height, img_width),
  batch_size=batch_size)

### Model Training

In [ ]:
ds = train_ds.concatenate(val_ds)

lookup_images = {}
lookup_labels = {}
for i, (x, y) in enumerate(ds):
  lookup_images[i] = x
  lookup_labels[i] = y

kfold = KFold(n_splits=5, shuffle=True)

for train, test in kfold.split(np.arange(len(ds))):
    images_train = np.concatenate(list(map(lookup_images.get, train)))
    labels_train = np.concatenate(list(map(lookup_labels.get, train)))
    images_test = np.concatenate(list(map(lookup_images.get, test)))
    labels_test = np.concatenate(list(map(lookup_labels.get, test)))
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3))])
    base_model = VGG16(include_top=False,
                       weights=None, input_shape=(224, 224, 3))
    model.add(base_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(1.0e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    history = model.fit(images_train, labels_train,batch_size= 128,
                        epochs=30,verbose=1)
    model.save('./emd_check.h5',save_format='h5')
    scores = model.evaluate(images_test, labels_test, verbose=1)